In [ ]:
import numpy as np
import scipy.io
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import ipywidgets as widgets
from IPython.display import display

In [ ]:
data_folder = "../../data"
n_buses = 118
num_samples = 30000

In [ ]:
path_to_data = data_folder + "/pfdelta/30000opfv2dataset_case118_ieee.mat"
pfdelta_data = scipy.io.loadmat(path_to_data)
baseMVA = pfdelta_data["baseMVA"]

num_samples = pfdelta_data["Dem"].shape[1]
num_buses = pfdelta_data["bus"].shape[0]

Pd = np.real(pfdelta_data["Dem"]).flatten(order="F")
Qd = np.imag(pfdelta_data["Dem"]).flatten(order="F")
Pg_at_gens = np.real(pfdelta_data["Gen"])
Qg_at_gens = np.imag(pfdelta_data["Gen"])
Vm = np.abs(pfdelta_data["Vol"]).flatten(order="F")
Va = np.angle(pfdelta_data["Vol"]).flatten(order="F")
Va = np.rad2deg(Va)

gen_bus = pfdelta_data["gen"][:, 0].astype(int) - 1

Pg = np.zeros((num_buses, num_samples))
Qg = np.zeros((num_buses, num_samples))

Pg[gen_bus, :] = Pg_at_gens
Qg[gen_bus, :] = Qg_at_gens

Pg = Pg.flatten(order="F")
Qg = Qg.flatten(order="F")

df_delta = pd.DataFrame(
    {
        "bus": np.tile(np.arange(num_buses), num_samples),
        "scenario": np.repeat(np.arange(num_samples), num_buses),
        "Pd": Pd,
        "Qd": Qd,
        "Pg": Pg,
        "Qg": Qg,
        "Vm": Vm,
        "Va": Va,
    }
)

In [ ]:
pg_learn = pd.read_csv(data_folder + "/PGLearn-Small-118_ieee-nminus1/pglearn.csv")

In [ ]:
opf_data = pd.read_csv(
    data_folder + "/opf_data_n_minus_one/case118_ieee/raw/bus_data.csv"
)

In [ ]:
versions = {}
versions["gridfm_datakit_pf"] = (
    data_folder + "/baseline_perturbations/case118_ieee/raw/bus_data.csv"
)
versions["gridfm_datakit_pf_lines_only"] = (
    data_folder + "/baseline_perturbations_lines_only/case118_ieee/raw/bus_data.csv"
)
versions["gridfm_datakit_opf"] = (
    data_folder + "/opf_baseline_perturbations_lines_only/case118_ieee/raw/bus_data.csv"
)

# add pfdelta
versions["pfdelta"] = df_delta
# add opf
versions["opf"] = opf_data
versions["pglearn"] = pg_learn

In [ ]:
description = {
    "gridfm_datakit_pf": "gridfm_datakit_pf",
    "opf": "OPF-Data",
    "pfdelta": "pfdelta",
    "gridfm_datakit_opf": "gridfm_datakit_opf",
    "pglearn": "Pglearn",
    "gridfm_datakit_pf_lines_only": "gridfm_datakit_pf_lines_only",
}

In [ ]:
FIXED_BUSES = np.random.randint(0, n_buses, size=18)
FIXED_BUSES = np.concatenate((FIXED_BUSES, [68, 69, 84]))
FIXED_BUSES = np.unique(FIXED_BUSES)

# If some duplicates were removed, add random buses to reach 22
while len(FIXED_BUSES) < 20:
    candidate = np.random.randint(0, n_buses)
    if candidate not in FIXED_BUSES:
        FIXED_BUSES = np.append(FIXED_BUSES, candidate)

In [ ]:
def load_csv(path):
    df = pd.read_csv(path)
    # for col in ["Pd", "Qd", "Pg", "Qg"]:
    #     df[col] = df[col] / 100.0
    return df


ALL_DFS = {
    lab: (load_csv(obj) if isinstance(obj, str) else obj)
    for lab, obj in versions.items()
}
dfs = {
    lab: df[df.scenario.isin(np.random.randint(0, num_samples, size=df.scenario.max()))]
    for lab, df in ALL_DFS.items()
}

In [ ]:
def plot_versions(feature, selected_versions):
    REFERENCE_VERSION = (
        "gridfm_datakit_pf"
        if "gridfm_datakit_pf" in selected_versions
        else "gridfm_datakit_opf"
    )
    grouped = {
        lab: df.groupby("bus")[feature]
        for lab, df in dfs.items()
        if lab in selected_versions
    }
    bus_types = (
        dfs[REFERENCE_VERSION].groupby("bus")[["PQ", "PV", "REF"]].first().values
    )
    bus_types = np.array(
        ["PQ" if t[0] == 1 else "PV" if t[1] == 1 else "REF" for t in bus_types]
    )

    fig, ax = plt.subplots(figsize=(12, 6))

    labels = list(grouped.keys())
    m = len(FIXED_BUSES)
    k = len(labels)
    base_positions = range(m)
    width = 0.7 / max(k, 1)
    offsets = [(i - (k - 1) / 2.0) * width for i in range(k)]
    COLORS = ["red", "blue", "green", "orange", "purple", "brown"]

    for i, lab in enumerate(labels):
        per_bus_data = [grouped[lab].get_group(b).values for b in FIXED_BUSES]
        positions = [p + offsets[i] for p in base_positions]

        bp = ax.boxplot(
            per_bus_data,
            positions=positions,
            widths=width * 0.9,
            showfliers=False,
            patch_artist=True,
            medianprops=dict(color="black", linewidth=1.5),  # median line stays black
            whis=(0.05, 0.95),
        )
        for patch in bp["boxes"]:
            patch.set_facecolor(COLORS[i % len(COLORS)])
            patch.set_alpha(0.5)

    # ax.set_title(f"{feature} distribution across buses", fontsize=14)
    ax.set_ylabel(feature, fontsize=14)
    ax.set_xticks(list(base_positions))
    ax.set_xticklabels(
        [f"Bus {b} {bus_types[b]}" for b in FIXED_BUSES],
        rotation=45,
        ha="right",
        fontsize=14,
    )

    ax.grid(True, axis="y", alpha=0.3)
    legend_patches = [
        mpatches.Patch(color=COLORS[i % len(COLORS)], label=description[lab])
        for i, lab in enumerate(labels)
    ]
    ax.legend(
        handles=legend_patches, title="Dataset version", loc="upper right", fontsize=12
    )

    plt.tight_layout()
    # plt.savefig(f"boxplot_{feature}_{"_".join(selected_versions)}.pdf")
    plt.show()


def plot_spider(selected_versions):
    features = ["Vm", "Va", "Pd", "Qd", "Pg", "Qg"]
    labels = list(selected_versions)
    COLORS = ["red", "blue", "green", "orange", "purple", "brown"]

    df_less_scenarios = {
        lab: df[
            df.scenario.isin(np.random.randint(0, num_samples, size=df.scenario.max()))
        ]
        for lab, df in ALL_DFS.items()
        if lab in selected_versions
    }

    grouped_std = {
        lab: df.groupby("bus")[features].std() for lab, df in df_less_scenarios.items()
    }
    grouped_mean = {
        lab: df.groupby("bus")[features].mean() for lab, df in df_less_scenarios.items()
    }

    stats = {}
    for lab in labels:
        vals = []
        for f in features:
            if f == "Va":
                grouped_std[lab][f] = np.deg2rad(grouped_std[lab][f])
                grouped_mean[lab][f] = np.deg2rad(grouped_mean[lab][f])
            mean_std = grouped_std[lab][f].mean()
            vals.append(mean_std)
        stats[lab] = vals

    df_stats = pd.DataFrame(stats).T
    df_stats.columns = features
    df_stats = df_stats.div(df_stats.max(axis=0), axis=1)

    N = len(features)
    angles = np.linspace(0, 2 * np.pi, N, endpoint=False).tolist()
    angles += angles[:1]

    fig, ax = plt.subplots(figsize=(7, 7), subplot_kw=dict(polar=True))

    for i, lab in enumerate(labels):
        values = df_stats.loc[lab].to_list()
        values += values[:1]
        ax.plot(
            angles,
            values,
            color=COLORS[i % len(COLORS)],
            linewidth=2,
            label=description[lab],
        )
        ax.fill(angles, values, color=COLORS[i % len(COLORS)], alpha=0.25)

    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(features, fontsize=14)
    # ax.set_title("Data diversity across datasets", y=1.1, fontsize=14)
    ax.grid(True)
    ax.legend(bbox_to_anchor=(1.3, 1.1), fontsize=12)
    plt.tight_layout()
    # plt.savefig("spider_plot.pdf")
    plt.show()


version_multi = widgets.SelectMultiple(
    options=[(description[k], k) for k in versions.keys()],  # label, value pairs
    value=list(versions.keys()),  # values must match the 2nd element of options
    description="Versions",
    rows=min(6, len(versions)),
    layout=widgets.Layout(width="300px"),
)

feature_toggle = widgets.ToggleButtons(
    options=["Vm", "Va", "Pd", "Qd", "Pg", "Qg"],
    value="Pg",
    description="Feature",
)


out = widgets.Output()


def _on_change(_=None):
    with out:
        out.clear_output(wait=True)
        if not version_multi.value:
            print("Select at least one dataset version.")
            return
        plot_versions(
            feature_toggle.value,
            list(version_multi.value),
        )
        plot_spider(
            list(version_multi.value),
        )


# controls setup
controls = widgets.HBox([version_multi, widgets.VBox([feature_toggle])])
display(controls, out)

# Observe widget changes
version_multi.observe(_on_change, names="value")
feature_toggle.observe(_on_change, names="value")